In [ ]:
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm
import re
import shutil
import datetime as dt
import numpy as np
from matplotlib import pyplot as plt
from netCDF4 import Dataset, date2index, num2date, date2num

In [ ]:
def list_files(directory):
    
    """
    List the content of a folder.

    Parameters
    ----------
    directory: string
        The name of the folder

    Returns
    -------
    Python object
        Files contained in the folder
    """

    e0 = '.nc'
    return (f for f in os.listdir(directory) if e0 in f)

Calculation of the yearly temperature trend from 2015 to 2100 over Europe.
The ensemble r5i1p1f1 was selected, but other can be chosen.
The temperture has been average over the whole Europe region (Lon -10 50 Lat 30 75).

In [ ]:
variant_label = 'r5i1p1f1'
year0 = 2015
yearf = 2100
target0 = {"lat": 30, "lon": 360-10}
targetf = {"lat": 75, "lon": 50}

new_folder = 'files_2015_2100'
useful_files = list_files(new_folder)

temp = np.empty((0, 0))
dates = np.empty((0, 0))
models = np.empty((0, 0), dtype = str)
for filename in useful_files:
    filename_path = new_folder + '/' + filename
    if variant_label in filename_path:
        nc = Dataset(filename_path)
        nc_time = nc.variables['time']
        time_start = date2index(dt.datetime(year0, 1, 1), nc_time, select="nearest")
        time_end = date2index(dt.datetime(yearf, 12, 31), nc_time, select="nearest")
        nc_lat = nc.variables['lat'][:].data
        nc_lon = nc.variables['lon'][:].data
        lat_cell0 = np.argmin(np.abs(nc_lat - target0['lat']))
        lat_cellf = np.argmin(np.abs(nc_lat - targetf['lat']))
        lon_cell0 = np.argmin(np.abs(nc_lon - target0['lon']))
        lon_cellf = np.argmin(np.abs(nc_lon - targetf['lon']))
        dat = nc.variables['ts'][time_start:(time_end + 1), lat_cell0:lat_cellf, lon_cell0:lon_cellf:-1]
        dat = np.subtract(dat.data, 273.15)
        dat_ave = np.mean(np.mean(dat, axis=2),axis=1)
        temp = np.append(temp, dat_ave) 
        datef = num2date(nc_time[time_start:time_end + 1], units=nc_time.units, calendar=nc_time.calendar)
        dates = np.append(dates, datef)
        subs = re.search(r'Amon_.+_ssp585', filename).group()
        modelf = np.chararray(dat_ave.shape, itemsize=len(subs[5:-7]))
        modelf[:] = subs[5:-7]
        models = np.append(models, modelf)

Plot of the yearly temperature trend from 2015 to 2100 over Europe. In this case the selected data contained two models. The script can plot data for an arbitrary number of models.

In [ ]:
data_models = []
for i in range(0,len(models)):
    if models[i] not in data_models:
        data_models.append(models[i])

fig = plt.figure(figsize=(10,4))
for mod in data_models:
    date_model = dates[models == mod]
    temp_model = temp[models == mod]
    ind = np.argsort(date_model)
    date_model = date_model[ind]
    temp_model = temp_model[ind]
    new_dates = [dt.date(i.year, i.month, i.day) for i in date_model]
    plt.plot(new_dates,temp_model,label = mod)
plt.xlabel("dates", fontsize=20)
plt.ylabel("Temperature [C]", fontsize=20)
plt.legend(fontsize=20)
plt.tick_params(labelsize=20)
plt.show()

fig.savefig('Yearly_temperature_Europe_ssp585.png', bbox_inches='tight')